# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator #for data preprocessing

In [2]:
tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
#Need to preprocess the training set to avoid overfitting. Otherwise, there will be a huge difference betweent the accuracy of training set and testing set.
#Hence we need to apply transformations. (for computer vision the way to avoid overfitting is to apply transformation)
#Some of geometrical transformations are to sift some of the pixels, rotate a bit of images, horizontal flips, zoom in and zoom out. The technical term refer to all this transformation is image augmentation.

train_datagen=ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True) #creating train_dategen object as an instance of the ImageDataGenerator class

rescale is used for feature scaling (here, applied on each and every pixel by dividing theri value by 255, which is just like normalization). Feature scaling is very compulsory for training neural networks.
<br>
We now need to connect the train_datagen object to our training dataset images.

In [4]:
training_set=train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)
#flow_from_directory connects the image augmentation tool to images from our training set.
#target_size is the final size of the images when they will be fed into the CNN.
#batch_size=how many images in each batch.
#class_mode= binary, since we have only binary output here Cat/Dog.

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen=ImageDataGenerator(rescale=1./255) # no transformation applied, only feature scaling done
test_set=test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn=tf.keras.models.Sequential()

### Step 1 - Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3, activation='relu', input_shape=(64,64,3)))
#kernel_size=size of feature detector
#3 in input shape is for colored image, if black & White then use 1.

### Step 2 - Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3, activation='relu', input_shape=(64,64,3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu')) #we might get better accuracy in the end with a larger number of neurons.


### Step 5 - Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [13]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x = training_set, validation_data=test_set, epochs=25)

Epoch 1/25


250/250 [==============================] - 251s 978ms/step - loss: 0.6619 - accuracy: 0.5979 - val_loss: 0.7106 - val_accuracy: 0.5830
Epoch 2/25
250/250 [==============================] - 69s 276ms/step - loss: 0.5982 - accuracy: 0.6862 - val_loss: 0.5489 - val_accuracy: 0.7265
Epoch 3/25
250/250 [==============================] - 55s 219ms/step - loss: 0.5666 - accuracy: 0.7015 - val_loss: 0.5554 - val_accuracy: 0.7290
Epoch 4/25
250/250 [==============================] - 51s 203ms/step - loss: 0.5379 - accuracy: 0.7293 - val_loss: 0.5138 - val_accuracy: 0.7560
Epoch 5/25
250/250 [==============================] - 49s 195ms/step - loss: 0.5080 - accuracy: 0.7479 - val_loss: 0.5068 - val_accuracy: 0.7525
Epoch 6/25
250/250 [==============================] - 48s 193ms/step - loss: 0.4897 - accuracy: 0.7600 - val_loss: 0.5061 - val_accuracy: 0.7570
Epoch 7/25
250/250 [==============================] - 48s 193ms/step - loss: 0.4789 - accuracy: 0.7684 - val_loss: 0.4629 - val

## Part 4 - Making a single prediction

In [15]:
import numpy as np
from keras.preprocessing import image
test_image=image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64,64))
# Converting the test_image (PIL format) to numpy array format
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image/255.0) #normalize the images
training_set.class_indices
if result[0][0] > 0.5:
  prediction='dog'
else:
  prediction='cat'


1/1 [==============================] - 1s 910ms/step


In [16]:
print(prediction)

dog
